In [1]:
import numpy as np
import cvxpy as cp

from problems.data_generation import generate_problem_data
from problems.problem_definition import BilevelProblem
from algorithms.barrier_blo_box import BarrierBLO
from algorithms.blocc import BLOCC
from algorithms.implicit_gradient_descent import IGD
from algorithms.BiC_GAFFA import BiC_GAFFA
from algorithms.BSG import BSG

n = 60
seed = 1

data = generate_problem_data(n, seed)
problem = BilevelProblem(data)

max_elapsed_time = 60

x_init = np.zeros(n)
y_init = np.zeros(n)
z_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
theta_init = np.zeros(n)
y_g_init = np.zeros(n)
y_F_init = np.zeros(n)
mu_g_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
mu_F_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
hparams = {
        'barrier_blo': {
            'M': 0.001,
            't': 0.01,
            'alpha_x': 0.002,
            'alpha_y': 0.1,
            'beta_y': 1,
            'epsilon_x': 0.1,
            'epsilon_y': 0.01,
            'inner_max_iters': 1000000,
            'outer_max_iters': 100000
        },
        'blocc': {
            'gamma': 2,
            'alpha_x': 0.000005,
            'alpha_g_y': 0.00001,
            'alpha_F_y': 0.00001,
            'beta_g_y': 20,
            'beta_F_y': 20,
            'epsilon_x': 0.1,
            'epsilon_inner_y_g': 0.01,
            'epsilon_outer_y_g': 0.01,
            'epsilon_inner_y_F': 0.01,
            'epsilon_outer_y_F': 0.01,
            'maxmin_g_outer_max_iters': 5000000,
            'maxmin_F_outer_max_iters': 5000000,
            'maxmin_g_inner_max_iters': 5000000,
            'maxmin_F_inner_max_iters': 5000000,
            'main_max_iters': 1000
        },
        'bic_gaffa':{
            'alpha':0.001,
            'c':1,
            'tau':1.3,
            'gamma_1':10,
            'gamma_2':1,
            'eta':1,
            'r':1,
            'epsilon':0.1,
            'max_iters':300
        },
        'BSG':{
            'alpha_x':0.00001,
            'alpha_y':0.00001,
            'alpha_z':0.00001,
            'epsilon_x':0.1,
            'epsilon_y':0.1,
            'epsilon_z':0.1,
            'max_iters_x':1000000,
            'max_iters_y':1000000,
            'max_iters_z':1000000
        },
        'IGD': {
            'M': 1e-3,
            't': 1e-3,
            'alpha_x': 0.001,
            'alpha_y': 0.001,
            'epsilon_x': 1e-3,
            'epsilon_y': 1e-3,
            'inner_max_iters': 1000,
            'outer_max_iters': 1000
        }
    }

# BSG Algorithm

In [5]:
bsg_algo = BSG(problem, hparams)

x_opt_bsg, y_opt_bsg, history = bsg_algo.bsg(x_init, y_init, z_init)
print(problem.f(x_opt_bsg, y_opt_bsg))

f(x, y) = 95.34565243294612, grad_norm of hyperfunction= 68.18518960114362
f(x, y) = 95.3797668458633, grad_norm of hyperfunction= 68.26745807853314
f(x, y) = 95.40775483344689, grad_norm of hyperfunction= 68.32573208356472
f(x, y) = 95.43200883750767, grad_norm of hyperfunction= 68.37336728405475
f(x, y) = 95.45384948221346, grad_norm of hyperfunction= 68.41466553631169
f(x, y) = 95.4738730959845, grad_norm of hyperfunction= 68.45163740620116
f(x, y) = 95.49258305564182, grad_norm of hyperfunction= 68.48542126690229
f(x, y) = 95.50997637053099, grad_norm of hyperfunction= 68.51671279800001
f(x, y) = 95.52649161605001, grad_norm of hyperfunction= 68.5460075094824
f(x, y) = 95.54226260369578, grad_norm of hyperfunction= 68.57364850096664
f(x, y) = 95.55726547213476, grad_norm of hyperfunction= 68.5998825438868
f(x, y) = 95.57170889311199, grad_norm of hyperfunction= 68.62491324566984
f(x, y) = 95.58565775958208, grad_norm of hyperfunction= 68.64889450071306
f(x, y) = 95.59916514602432, 

KeyboardInterrupt: 

# BiC_GAFFA Algorithm

In [9]:
bic_algo = BiC_GAFFA(problem, hparams)

x_opt_bic, y_opt_bic, history = bic_algo.bic_gaffa(x_init, y_init, z_init, theta_init)
print(problem.f(x_opt_bic, y_opt_bic))

f(x,y)=-9.747361573824344
f(x,y)=-14.51712463759706
f(x,y)=-17.1834466628215
f(x,y)=-17.762133683784555
f(x,y)=-17.73259788745306
end: norm of y 0.3562318104870582
-17.73259788745306


# BiC_GAFFA Feasibility Check

In [10]:
barrier_algo = BarrierBLO(problem, hparams)

h1_values = [problem.h_1(x_opt_bic, y_opt_bic, i) for i in range(problem.num_constraints_h1)]
h2_values = [problem.h_2(x_opt_bic, y_opt_bic, i) for i in range(problem.num_constraints_h2)]

y_original_opt_blocc = barrier_algo.Interior_inner_loop(x_opt_bic, y_opt_bic)
print(f"f_theoretical_opt_result={problem.f(x_opt_bic, y_original_opt_blocc)}, f_blocc_numerical_result={problem.f(x_opt_bic, y_opt_bic)}")

print(f"g_theoretical_opt_result={problem.g(x_opt_bic, y_original_opt_blocc)}, g_blocc_numerical_result={problem.g(x_opt_bic, y_opt_bic)}")

for i, h_val in enumerate(h1_values):
    print(f"h_1[{i}] = {h_val}")

for i, h_val in enumerate(h2_values):
    print(f"h_2[{i}] = {h_val}")

f_theoretical_opt_result=82.43574530825444, f_blocc_numerical_result=-17.73259788745306
g_theoretical_opt_result=-47.69613326684787, g_blocc_numerical_result=-19.286098699648562
h_1[0] = -0.17400680845175703
h_1[1] = -0.6632705965967381
h_1[2] = -0.3829367915749853
h_1[3] = -0.9341368817662237
h_1[4] = -0.02482701413245024
h_1[5] = -0.1089054297468687
h_1[6] = -0.5322507371751827
h_1[7] = -0.029946131876903162
h_1[8] = -0.47690740289258055
h_1[9] = -0.342451016789516
h_1[10] = -0.1882337467696425
h_1[11] = -0.8530285560907548
h_1[12] = -0.22864600718461736
h_1[13] = -0.761392200237247
h_1[14] = -0.710045812879819
h_1[15] = -0.18223009433219511
h_1[16] = 0.0
h_1[17] = -0.45935732407551055
h_1[18] = -0.837088132733038
h_1[19] = -0.7285203753263796
h_2[0] = -0.9587785177376646
h_2[1] = -1.0677335886501895
h_2[2] = -0.9441114146957
h_2[3] = -0.9531779721829531
h_2[4] = -1.0613064759013158
h_2[5] = -1.053626629071047
h_2[6] = -1.0028640781308045
h_2[7] = -0.9918228236584675
h_2[8] = -0.9625

# BLOCC Algorithm

In [13]:
blocc_algo = BLOCC(problem, hparams)


x_opt_blocc, y_opt_blocc, history = blocc_algo.blocc(x_init, y_g_init, y_F_init, mu_g_init, mu_F_init)
print(problem.f(x_opt_blocc, y_opt_blocc))



Main iteration 1
Inner loop for L_g converges when iter = 16516
Outer loop for L_g converges when iter = 0
Inner loop for L_F converges when iter = 4230
Outer loop for L_F converges when iter = 0
f(x,y)=4.033321485990886,grad_norm = 66.51969461261397
Main iteration 2
Inner loop for L_g converges when iter = 15274
Outer loop for L_g converges when iter = 0
Inner loop for L_F converges when iter = 4230
Outer loop for L_F converges when iter = 0
f(x,y)=4.011325551351373,grad_norm = 66.47946533329478
Main iteration 3
Inner loop for L_g converges when iter = 15274
Outer loop for L_g converges when iter = 0
Inner loop for L_F converges when iter = 4230
Outer loop for L_F converges when iter = 0
f(x,y)=3.98935633679546,grad_norm = 66.43914095175064
Main iteration 4
Inner loop for L_g converges when iter = 15274
Outer loop for L_g converges when iter = 0
Inner loop for L_F converges when iter = 4230
Outer loop for L_F converges when iter = 0
f(x,y)=3.9674137583416424,grad_norm = 66.39884114943

KeyboardInterrupt: 

# BLOCC Feasibility Check

In [6]:
barrier_algo = BarrierBLO(problem, hparams)

h1_values = [problem.h_1(x_opt_blocc, y_opt_blocc, i) for i in range(problem.num_constraints_h1)]
h2_values = [problem.h_2(x_opt_blocc, y_opt_blocc, i) for i in range(problem.num_constraints_h2)]

y_original_opt_blocc = barrier_algo.Interior_inner_loop(x_opt_blocc, y_opt_blocc)
print(f"f_theoretical_opt_result={problem.f(x_opt_blocc, y_original_opt_blocc)}, f_blocc_numerical_result={problem.f(x_opt_blocc, y_opt_blocc)}")

print(f"g_theoretical_opt_result={problem.g(x_opt_blocc, y_original_opt_blocc)}, g_blocc_numerical_result={problem.g(x_opt_blocc, y_opt_blocc)}")

for i, h_val in enumerate(h1_values):
    print(f"h_1[{i}] = {h_val}")

for i, h_val in enumerate(h2_values):
    print(f"h_2[{i}] = {h_val}")

f_theoretical_opt_result=-10.022642705440504, f_blocc_numerical_result=-19.398774025508203
g_theoretical_opt_result=-14.356151349132334, g_blocc_numerical_result=-13.670580732825421
h_1[0] = -0.13806672090371802
h_1[1] = -0.671302820901845
h_1[2] = -0.3464573017595583
h_1[3] = -0.9484930893558797
h_1[4] = 0.009926023757235308
h_1[5] = -0.18425521995274768
h_1[6] = -0.5900273891286644
h_1[7] = -0.0009323411139185306
h_1[8] = -0.36200292908214393
h_1[9] = -0.49765119732566215
h_1[10] = -0.23749131289168957
h_1[11] = -0.957581425212053
h_1[12] = -0.30805960427026663
h_1[13] = -0.6233170934976497
h_1[14] = -0.5781179015201512
h_1[15] = -0.1324298998154661
h_1[16] = -0.06990942539075545
h_1[17] = -0.42808115272822045
h_1[18] = -0.9295125312430059
h_1[19] = -0.7524010872744005
h_2[0] = -0.9900552107555037
h_2[1] = -1.0621903947111715
h_2[2] = -0.8778618740740134
h_2[3] = -0.917031854822307
h_2[4] = -1.1059287820400316
h_2[5] = -1.0620434906645924
h_2[6] = -1.0104549014865363
h_2[7] = -0.9877

# Implicit Gradient Descent Algorithm

In [49]:
IGD_algo = IGD(problem, hparams)

x_opt_IGD, y_opt_IGD, history = IGD_algo.upper_loop(x_init, y_init)

print("Optimized x:", x_opt_IGD)
print("Optimized y:", y_opt_IGD)

h1_values = [problem.h_1(x_opt_IGD, y_opt_IGD, i) for i in range(problem.num_constraints_h1)]

print("\nValues of h_1 constraints at the optimal point:")
for i, h_val in enumerate(h1_values):
    print(f"h_1[{i}] = {h_val}")

times = [h['time'] for h in history]
grad_norms = [h['grad_norm'] for h in history]

Outer iteration 1
Inner loop converged at iteration 200
f(x, y) = 95.45037851360806, grad_norm = 76.59673418115331
Outer iteration 2
Inner loop converged at iteration 112
f(x, y) = 91.19500851740221, grad_norm = 67.2257525452191
Outer iteration 3
Inner loop converged at iteration 110
f(x, y) = 87.9160198480594, grad_norm = 59.00630049516197
Outer iteration 4
Inner loop converged at iteration 107
f(x, y) = 85.39015297241798, grad_norm = 51.796437571852486
Outer iteration 5
Inner loop converged at iteration 105
f(x, y) = 83.44417540097366, grad_norm = 45.471557030758525
Outer iteration 6
Inner loop converged at iteration 102
f(x, y) = 81.94475905747844, grad_norm = 39.922553604489096
Outer iteration 7
Inner loop converged at iteration 100
f(x, y) = 80.7892951506114, grad_norm = 35.053784045327106
Outer iteration 8
Inner loop converged at iteration 98
f(x, y) = 79.8987827656407, grad_norm = 30.781487703596458
Outer iteration 9
Inner loop converged at iteration 95
f(x, y) = 79.212391899265

# Implicit Gradient Descent Feasibility Check

In [50]:
barrier_algo = BarrierBLO(problem, hparams)

h1_values = [problem.h_1(x_opt_IGD, y_opt_IGD, i) for i in range(problem.num_constraints_h1)]
h2_values = [problem.h_2(x_opt_IGD, y_opt_IGD, i) for i in range(problem.num_constraints_h2)]

y_original_opt_IGD = barrier_algo.Interior_inner_loop(x_opt_IGD, y_opt_IGD)
print(f"f_theoretical_opt_result={problem.f(x_opt_IGD, y_original_opt_IGD)}, f_IGD_numerical_result={problem.f(x_opt_IGD, y_opt_IGD)}")

print(f"g_theoretical_opt_result={problem.g(x_opt_IGD, y_original_opt_IGD)}, g_IGD_numerical_result={problem.g(x_opt_IGD, y_opt_IGD)}")

for i, h_val in enumerate(h1_values):
    print(f"h_1[{i}] = {h_val}")

for i, h_val in enumerate(h2_values):
    print(f"h_2[{i}] = {h_val}")

f_theoretical_opt_result=59.04726374415966, f_IGD_numerical_result=76.90380080947456
g_theoretical_opt_result=-47.905074255466786, g_IGD_numerical_result=-51.811093935401786
h_1[0] = -1.818606806017621
h_1[1] = -0.6332653285456722
h_1[2] = 0.692060467876725
h_1[3] = -0.19201660481909777
h_1[4] = -0.600298040133991
h_1[5] = -0.2737396315591377
h_1[6] = -1.012366935480417
h_1[7] = 0.4450296606186358
h_1[8] = -1.4287421110645298
h_1[9] = -0.8173446049192401
h_1[10] = 1.0874902560609827
h_1[11] = -1.4789022155643556
h_1[12] = 0.10827124470942368
h_1[13] = -1.786263928772887
h_1[14] = -0.8903993418731315
h_1[15] = -0.07504978497512865
h_1[16] = -0.3015547409733911
h_1[17] = -0.21462543837571568
h_1[18] = -0.693290361744195
h_1[19] = 0.5416734352752851
h_2[0] = -1.0929089554460576
h_2[1] = -0.9654870816489937
h_2[2] = -1.0237375559765334
h_2[3] = -1.0192583555543342
h_2[4] = -1.044948936664087
h_2[5] = -0.877464459734365
h_2[6] = -1.0860014094637798
h_2[7] = -1.2018995373962
h_2[8] = -1.4768

# BFBM Algorithm

In [3]:
barrier_algo = BarrierBLO(problem, hparams)

# y_proj = barrier_algo.project_to_constraints(x_init, y_init)

# h = problem.h_1(x_init, y_proj, 0)

# print(y_proj)

# print(h)



x_opt_barrier, y_opt_barrier, history = barrier_algo.upper_loop(x_init, y_init, max_elapsed_time)

print("Optimized x:", x_opt_barrier)
print("Optimized y:", y_opt_barrier)

h1_values = [problem.h_1(x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h1)]

print("\nValues of h_1 constraints at the optimal point:")
for i, h_val in enumerate(h1_values):
    print(f"h_1[{i}] = {h_val}")

times = [h['time'] for h in history]
grad_norms = [h['grad_norm'] for h in history]

Outer iteration 1
Inner loop converged at iteration 85
f(x, y) = 87.83830663751468, grad_norm of hyperfunction= 69.85153610048881
Outer iteration 2
Inner loop converged at iteration 64
f(x, y) = 106.4256678530625, grad_norm of hyperfunction= 468.23831249856966
Outer iteration 3
Inner loop converged at iteration 92
f(x, y) = 29.412840550297652, grad_norm of hyperfunction= 146.50921630211204
Outer iteration 4
Inner loop converged at iteration 88
f(x, y) = 12.184278453991865, grad_norm of hyperfunction= 119.95219349457118
Outer iteration 5
Inner loop converged at iteration 81
f(x, y) = 2.5165049612467527, grad_norm of hyperfunction= 91.08561498065795
Outer iteration 6
Inner loop converged at iteration 79
f(x, y) = -2.8350404958491353, grad_norm of hyperfunction= 68.81029891795728
Outer iteration 7
Inner loop converged at iteration 80
f(x, y) = -5.80707805147631, grad_norm of hyperfunction= 56.40704692265707
Outer iteration 8
Inner loop converged at iteration 79
f(x, y) = -7.79761543298420

# BFBM Feasibility Check

In [3]:
barrier_algo = BarrierBLO(problem, hparams)

h1_values = [problem.h_1(x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h1)]
h2_values = [problem.h_2(x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h2)]

y_original_opt_bfbm = barrier_algo.Interior_inner_loop(x_opt_barrier, y_opt_barrier)
print(f"f_theoretical_opt_result={problem.f(x_opt_barrier, y_original_opt_bfbm)}, f_bfbm_numerical_result={problem.f(x_opt_barrier, y_opt_barrier)}")

print(f"g_theoretical_opt_result={problem.g(x_opt_barrier, y_original_opt_bfbm)}, g_bfbm_numerical_result={problem.g(x_opt_barrier, y_opt_barrier)}")

for i, h_val in enumerate(h1_values):
    print(f"h_1[{i}] = {h_val}")

for i, h_val in enumerate(h2_values):
    print(f"h_2[{i}] = {h_val}")

NameError: name 'x_opt_barrier' is not defined